In [32]:
from transformers import RobertaModel, RobertaModel, RobertaTokenizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import torch
import glob
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from scipy.stats import entropy


In [33]:
# AD train
AD_train=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\Data\AD"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df1 = pd.read_csv(f)
    df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
    AD_train.append(df1)


df_AD_train=pd.concat(AD_train,axis=0,ignore_index=True)
label_AD=0 #AD : 0 label
df_AD_train['label']=label_AD

In [34]:
# AD_Test
AD_test=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\Data\AD\AD_Testing"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df1 = pd.read_csv(f)
    df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
    AD_test.append(df1)


df_AD_test=pd.concat(AD_test,axis=0,ignore_index=True)
label_AD=0 #AD : 0 label
df_AD_test['label']=label_AD

In [35]:
# HC Train
HC_train=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\Data\HC"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df2=pd.read_csv(f)
    df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
    HC_train.append(df2)


df_HC_train=pd.concat(HC_train,axis=0,ignore_index=True)
label_HC=1 #HC : 1 label
df_HC_train['label']=label_HC


In [36]:
# HC Test
HC_test=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\Data\HC\HC_Testing"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df2=pd.read_csv(f)
    df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
    HC_test.append(df2)


df_HC_test=pd.concat(HC_test,axis=0,ignore_index=True)
label_HC=1 #HC : 1 label
df_HC_test['label']=label_HC

In [37]:
df_train=pd.concat([df_AD_train,df_HC_train], ignore_index=True)
df_test=pd.concat([df_AD_test,df_HC_test], ignore_index=True)

In [38]:
df_train = df_train.sample(frac=1,random_state=0).reset_index(drop=True)
df_test = df_test.sample(frac=1,random_state=0).reset_index(drop=True)
print('length of train', len(df_train),'length of test', len(df_test))

length of train 1968 length of test 649


In [39]:
tokenizer=RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base',output_hidden_states=True)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
sentences_train=df_train['sentence']
sentences_test=df_test['sentence']
total_sentences={'sentences_train':sentences_train,'sentences_test':sentences_test}

In [41]:
input_id_train=[]
input_id_test=[]
max_len = 47

for key in total_sentences.keys():
    input_ids = [torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)) for sentence in total_sentences[key]]

# Pad the sequences to have the same length
    
    padded_input = [torch.cat((input_id, torch.zeros((max_len - len(input_id)), dtype=torch.long))) for input_id in input_ids]
    padded_input = torch.stack(padded_input).to(torch.int64)

    if key=='sentences_train':
        input_id_train.append(padded_input)
    else :
        input_id_test.append(padded_input)
    

In [42]:
input_id_train=input_id_train[0]
input_id_test=input_id_test[0]

In [43]:
with torch.no_grad():
    output = model(input_id_train)
    last_hidden_train = output[2] #input for next part like forward pass

In [44]:
with torch.no_grad():
    output = model(input_id_test)
    last_hidden_test = output[2]

In [45]:
'''
shape of single word encoder: [2617,47,768], each word: [1,768],
each sentence: [1,47,768] shape.
'''
embedding_train=[]
embedding_test=[]
hidden_state_pool={'hidden_train':last_hidden_train, 'hidden_test':last_hidden_test}

for key in hidden_state_pool.keys():
    x=torch.stack(hidden_state_pool[key][-6:]).sum(0)
    print(torch.min(x))
    x=x+torch.max(x) # deviation from negative to positive 
    print(x.shape)
    # normalized=normalize(x, p=1, dim=0)
    # print(normalized.shape)
    sentences_embedding = entropy(x, axis=1) 
    print(sentences_embedding.shape) # single sentence of length 768

    if key=='hidden_train':
        embedding_train.append(sentences_embedding)
    else:
        embedding_test.append(sentences_embedding)


tensor(-74.2902)
torch.Size([1968, 47, 768])
(1968, 768)
tensor(-75.7942)
torch.Size([649, 47, 768])
(649, 768)


In [46]:
embedding_train[0].min()

3.8133354

In [50]:
x_train=embedding_train[0]
x_test = embedding_test[0]
y_train=df_train['label']
y_test=df_test['label']

# model(GaussianProcessClassifier result is 66% through combination of last 6 encoders)
model = GaussianProcessClassifier(kernel=1.0 * RBF(0.7),random_state=0,max_iter_predict=500)


model.fit(x_train,y_train)


GaussianProcessClassifier(kernel=1**2 * RBF(length_scale=0.7),
                          max_iter_predict=500, random_state=0)

In [51]:
y_pred=model.predict(x_test)
print(f"print score for :{metrics.accuracy_score(y_test,y_pred)}")

# accuracy lies between 0.60 - 0.66

print score for :0.6517719568567026


In [ ]:
# class_labels=  model.classes_
# accuracy_matrix = confusion_matrix(y_test, y_pred)



In [ ]:
# AD : 0, HC : 1
class_labels=  [0,1]
print(classification_report(y_test, y_pred, labels=class_labels)) 

              precision    recall  f1-score   support

           0       0.54      0.76      0.63       348
           1       0.48      0.25      0.33       301

    accuracy                           0.53       649
   macro avg       0.51      0.51      0.48       649
weighted avg       0.51      0.53      0.49       649

